This file is to analyze some basic characteristics of the "Index Event Data" file

In [2]:
#Setup
import pandas as pd
import yfinance as yf
import logging
import time
import re
from concurrent.futures import ThreadPoolExecutor

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load the Excel file
#file_path = 'https://www.dropbox.com/scl/fi/j2nocb5krdhn30uurjn4b/Index-Event-Data.xlsx?rlkey=n5gnmlgjwsqocp8heqd41jqoo&st=d3rjl5nf&dl=1'  # Adjust this path to your actual file location
file_path = 'D:\GitHub\Equity Project\Index Event Data.xlsx'
index_event_data = pd.read_excel(file_path, sheet_name='Data')

# 1 Indices and Actions

We first look at what kind of indices are featured in this file, and what kind of actions:

In [9]:
# Extract and sort the list of unique indices and actions
indices = sorted(index_event_data['Index Change'].unique())
cleaned_actions = sorted(index_event_data['Action'].unique())

# Create a two-way table (pivot table)
two_way_table = pd.pivot_table(index_event_data, values='Ticker', index='Index Change', columns='Action', aggfunc='count', fill_value=0)

two_way_table.loc['Total',:]= two_way_table.sum(axis=0)
two_way_table.loc[:,'Total'] = two_way_table.sum(axis=1)

# Display the two-way table
print("\nTwo-way table showing the count of entries for each action and index:")
two_way_table



Two-way table showing the count of entries for each action and index:


Action,Add,Delete,Downweight,Drop from SP600,MSCI SC to Std,MSCI Std to SC,Reclassification,SP400 to SP500,SP400 to SP600,SP500 to SP400,SP500 to SP600,SP600 to SP400,Upweight,Total
Index Change,,,,,,,,,,,,,,
MSCI SC,15.0,164.0,8.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0,446.0
MSCI Std,8.0,17.0,5.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,72.0
Nasdaq 100,14.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0
Russell 1000,25.0,33.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,86.0,149.0
Russell 2000,24.0,137.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0,263.0
Russell 3000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
S&P Completion,7.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,18.0
S&P Total Market,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
SP400,70.0,31.0,2.0,0.0,0.0,0.0,0.0,5.0,25.0,5.0,0.0,14.0,39.0,191.0


(Duplicates, such as "Add" and "Addition", are merged; one entry had action "z", changed to Upweight by context)
(We also note that the "Reclassification" action occurs only once, and created no share changes.)

# 2 Sectors and tickers

We now look at what sectors and tickers are featured in the file

In [38]:
# Extract and sort the list of unique tickers and sectors
tickers = index_event_data['Ticker'].unique()
sectors = sorted(index_event_data['Sector'].unique())

print("Number of Tickers: " + str(len(tickers)))
print("Sectors: "+ str(sectors))

Number of Tickers: 925
Sectors: ['Communication Services', 'Consumer Discretionary', 'Consumer Staples', 'Energy', 'Financials', 'Health Care', 'Industrials', 'Information Technology', 'Materials', 'Real Estate', 'Utilities']


(There also were duplicates in the "Sector" column, which has been consolidated)
(In one instance, the "Sector" value is "#NA". This has been patched with context clues)

Now we make a two-way table between Actions and Sectors:

In [40]:
# Create a two-way table (pivot table)
two_way_table = pd.pivot_table(index_event_data, values='Ticker', index='Sector', columns='Action', aggfunc='count', fill_value=0)

two_way_table.loc['Total',:]= two_way_table.sum(axis=0)
two_way_table.loc[:,'Total'] = two_way_table.sum(axis=1)

# Display the two-way table
print("\nTwo-way table showing the count of entries for each action and sector:")
two_way_table


Two-way table showing the count of entries for each action and sector:


Action,Add,Add to SP400,Add to SP500,Add to SP600,Delete,Downweight,Drop from SP600,MSCI SC to Std,MSCI Std to SC,Reclassification,SP400 to SP 600,SP400 to SP500,SP400 to SP600,SP500 to SP400,SP500 to SP600,SP600 to SP400,Upweight,Total
Sector,,,,,,,,,,,,,,,,,,
Communication Services,15.0,1.0,0.0,1.0,23.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,2.0,0.0,21.0,69.0
Consumer Discretionary,24.0,1.0,0.0,0.0,50.0,5.0,5.0,0.0,0.0,0.0,4.0,0.0,5.0,2.0,2.0,2.0,30.0,130.0
Consumer Staples,9.0,0.0,0.0,0.0,11.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,4.0,0.0,2.0,4.0,14.0,48.0
Energy,20.0,1.0,0.0,1.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,72.0,136.0
Financials,18.0,0.0,0.0,2.0,60.0,6.0,1.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0,0.0,2.0,83.0,180.0
Health Care,49.0,1.0,0.0,2.0,123.0,10.0,4.0,0.0,4.0,0.0,1.0,0.0,13.0,2.0,8.0,2.0,168.0,387.0
Industrials,26.0,2.0,0.0,0.0,63.0,5.0,0.0,0.0,0.0,0.0,3.0,4.0,10.0,1.0,1.0,4.0,67.0,186.0
Information Technology,35.0,1.0,1.0,0.0,80.0,3.0,3.0,0.0,0.0,0.0,1.0,2.0,14.0,0.0,2.0,4.0,54.0,200.0
Materials,10.0,3.0,0.0,0.0,18.0,0.0,2.0,0.0,0.0,1.0,1.0,2.0,3.0,0.0,0.0,2.0,29.0,71.0


With this, we now attempt to obtain the price and volumn data of the tickers around the events, as well as the  SPDR sectorial ETF during the time frame. 

# 3 Regular vs one-off events from Comments

In this section, we look at the kind of comments there are, and if they are informative of regular events or not

In [8]:
# Count occurrences in the "Comments" column
comment_counts = index_event_data['Comments'].value_counts()

# Sort the comments by occurrences
sorted_comments = comment_counts.sort_values(ascending=False)

# Display the sorted comments
sorted_comments_df = sorted_comments.to_frame(name='Occurrences')
sorted_comments_df.reset_index(inplace=True)
sorted_comments_df.columns = ['Shs to Trade', 'Occurrences']

# # Save the sorted comments to a CSV file
# sorted_comments_file_path = '/mnt/data/Sorted_Comments_By_Occurrences.csv'
# sorted_comments_df.to_csv(sorted_comments_file_path, index=False)

sorted_comments_df.head(20)


,Shs to Trade,Occurrences
0,Equity Offering,286
1,Quarterly Rebalance,51
2,Moved from SP400,30
3,Moved to SP600,27
4,Moved to SP400,20
5,Secondary Offering,18
6,Moved from SP600,17
7,Equity offering,16
8,Discretionary Removal,15
9,Public Offering,15
